In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.applications import  DenseNet121, VGG16, ResNet50, Xception, EfficientNetB7
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

tf.keras.backend.clear_session()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
dataset_dir = r'C:\Users\Metehan Sevgil\Desktop\KODLAR\Anaconda\Bitirme\lung_colon_image_set'

file_paths = []
labels = []

for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpeg'):
            file_paths.append(os.path.join(root, file))
            labels.append(os.path.basename(root))

data_df = pd.DataFrame({'filepaths': file_paths, 'labels': labels})
print(data_df.head())
print(data_df.tail())

                                           filepaths     labels
0  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
1  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
2  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
3  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
4  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
                                               filepaths    labels
24995  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24996  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24997  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24998  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24999  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc


In [27]:
max_epoch = 15
batch_size = 8
img_size = (256, 256)

tr_gen = ImageDataGenerator(rescale=1. / 255)
ts_gen = ImageDataGenerator(rescale=1. / 255)
vs_gen = ImageDataGenerator(rescale=1. / 255)

train_gen = tr_gen.flow_from_directory("./dataset/train",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

test_gen = ts_gen.flow_from_directory("./dataset/test",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

val_gen = vs_gen.flow_from_directory("./dataset/val",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

Found 15000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.


In [5]:
def build_model_resnet50(hp):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    base_model.trainable = True

    densenet_model = Sequential()
    densenet_model.add(base_model)

    densenet_model.add(Flatten())
    
    densenet_model.add(BatchNormalization())   
    densenet_model.add(Dense(units=hp.Choice('units', [128, 256, 512]), activation= 'relu'))
    densenet_model.add(Dropout(rate= hp.Choice('dropout_rate', [0.4, 0.5])))

    densenet_model.add(BatchNormalization())
    
    densenet_model.add(Dense(5, activation='softmax'))

    densenet_model.compile(optimizer=SGD(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    return densenet_model 

In [6]:
from  keras_tuner import HyperParameters

hp = HyperParameters()
hp.Choice('units', [128, 256, 512])
hp.Choice('dropout_rate', [0.4, 0.5])

0.4

In [7]:
densenet_model = build_model_resnet50(hp)

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
from keras_tuner.tuners import Hyperband

tuner = Hyperband(build_model_resnet50,
                  objective='val_loss',
                  max_epochs=5,
                  factor=2,   # Her iterasyonda modeli 1/2 oranında azaltır.             
                  directory='hyperband_tuning',
                  project_name='hyperband_densenet121_256')

In [22]:
tuner.search(train_gen,
             validation_data=val_gen,
             epochs=5)

Trial 11 Complete [00h 13m 13s]
val_loss: 0.05704978108406067

Best val_loss So Far: 0.02340041473507881
Total elapsed time: 02h 12m 34s


In [23]:
best_hps_densenet121_256 = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hps_densenet121_256.values) 

Best Hyperparameters:
{'units': 512, 'dropout_rate': 0.4, 'tuner/epochs': 3, 'tuner/initial_epoch': 2, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0003'}


In [24]:
best_model_densenet121_256 = tuner.hypermodel.build(best_hps_densenet121_256)

best_model_densenet121_256.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 8, 8, 1024)        7037504   
                                                                 
 flatten_1 (Flatten)         (None, 65536)             0         
                                                                 
 batch_normalization_2 (Batc  (None, 65536)            262144    
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 512)               33554944  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [28]:
history_densenet121_256 = best_model_densenet121_256.fit(train_gen,
                                                         validation_data=val_gen,
                                                         epochs=max_epoch,
                                                         callbacks=[early_stopping])

Epoch 1/15
1875/1875 [==============================] - 390s 208ms/step - loss: 0.3084 - accuracy: 0.8943 - val_loss: 0.1049 - val_accuracy: 0.9590
Epoch 2/15
1875/1875 [==============================] - 390s 208ms/step - loss: 0.1206 - accuracy: 0.9557 - val_loss: 0.0710 - val_accuracy: 0.9736
Epoch 3/15
1875/1875 [==============================] - 394s 210ms/step - loss: 0.0916 - accuracy: 0.9691 - val_loss: 0.0584 - val_accuracy: 0.9798
Epoch 4/15
1875/1875 [==============================] - 391s 209ms/step - loss: 0.0621 - accuracy: 0.9792 - val_loss: 0.0246 - val_accuracy: 0.9912
Epoch 5/15
1875/1875 [==============================] - 391s 209ms/step - loss: 0.0603 - accuracy: 0.9801 - val_loss: 0.0225 - val_accuracy: 0.9910
Epoch 6/15
1875/1875 [==============================] - 391s 208ms/step - loss: 0.0428 - accuracy: 0.9852 - val_loss: 0.0136 - val_accuracy: 0.9946
Epoch 7/15
1875/1875 [==============================] - 390s 208ms/step - loss: 0.0396 - accuracy: 0.9873 - val_

In [29]:
import json

with open("history_densenet121_256.json", "w") as f:
    json.dump(history_densenet121_256.history, f)

In [30]:
best_model_densenet121_256.save('densenet121_model_256.h5')